In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Responsible AI with Vertex AI Gemini API: Safety ratings and thresholds

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fresponsible-ai%2Fgemini_safety_ratings.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/responsible-ai/gemini_safety_ratings.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

| | |
|-|-|
|Author(s) | [Hussain Chinoy](https://github.com/ghchinoy) |

## Overview

Large language models (LLMs) can translate language, summarize text, generate creative writing, generate code, power chatbots and virtual assistants, and complement search engines and recommendation systems. The incredible versatility of LLMs is also what makes it difficult to predict exactly what kinds of unintended or unforeseen outputs they might produce.

Given these risks and complexities, the Vertex AI Gemini API is designed with [Google's AI Principles](https://ai.google/responsibility/principles/) in mind. However, it is important for developers to understand and test their models to deploy safely and responsibly. To aid developers, Vertex AI Studio has built-in content filtering, safety ratings, and the ability to define safety filter thresholds that are right for their use cases and business.

For more information, see the [Google Cloud Generative AI documentation on Responsible AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai) and [Configuring safety attributes](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes).

### Objectives

In this tutorial, you learn how to inspect the safety ratings returned from the Vertex AI Gemini API using the Python SDK and how to set a safety threshold to filter responses from the Vertex AI Gemini API.

The steps performed include:

- Call the Vertex AI Gemini API and inspect safety ratings of the responses
- Define a threshold for filtering safety ratings according to your needs

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK for Python


In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Define project information
PROJECT_ID = "[your project here]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


In [ ]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
)

### Load the Gemini 1.0 Pro model


In [ ]:
model = GenerativeModel("gemini-1.5-pro")

# Set parameters to reduce variability in responses
generation_config = GenerationConfig(
    temperature=0,
    top_p=0.1,
    top_k=1,
    max_output_tokens=1024,
)

## Generate text and show safety ratings

Start by generating a pleasant-sounding text response using Gemini.

In [ ]:
# Call Gemini API
nice_prompt = "Say three nice things about me"
responses = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

#### Inspecting the safety ratings

Look at the `safety_ratings` of the streaming responses.

In [11]:
responses = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "As"
    }
  }
}
usage_metadata {
}

candidates {
  content {
    role: "model"
    parts {
      text: " an AI, I don\'t know you personally, so I can\'t"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.1083984375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0693359375
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0517578125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.02099609375
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.1728515625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.09130859375
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.20703125
    severity: HARM_SEVERITY_NEGLIGIB

#### Understanding the safety ratings: category and probability

You can see the safety ratings, including each `category` type and its associated `probability` label, as well as a `probability_score`. Additionally, safety ratings have been expanded to `severity` and `severity_score`.

The `category` types include:

* Hate speech: `HARM_CATEGORY_HATE_SPEECH`
* Dangerous content: `HARM_CATEGORY_DANGEROUS_CONTENT`
* Harassment: `HARM_CATEGORY_HARASSMENT`
* Sexually explicit statements: `HARM_CATEGORY_SEXUALLY_EXPLICIT`

The `probability` labels are:

* `NEGLIGIBLE` - content has a negligible probability of being unsafe
* `LOW` - content has a low probability of being unsafe
* `MEDIUM` - content has a medium probability of being unsafe
* `HIGH` - content has a high probability of being unsafe

The `probability_score` has an associated confidence score between 0.0 and 1.0.

Each of the four safety attributes is assigned a safety rating (severity level) and a severity score ranging from 0.0 to 1.0, rounded to one decimal place. The ratings and scores in the following table reflect the predicted severity of the content belonging to a given category:


#### Comparing Probablity and Severity


There are two types of safety scores:

* Safety scores based on **probability** of being unsafe
* Safety scores based on **severity** of harmful content

The probability safety attribute reflects the likelihood that an input or model response is associated with the respective safety attribute. The severity safety attribute reflects the magnitude of how harmful an input or model response might be.

Content can have a low probability score and a high severity score, or a high probability score and a low severity score. For example, consider the following two sentences:

- The robot punched me.
- The robot slashed me up.

The first sentence might cause a higher probability of being unsafe and the second sentence might have a higher severity in terms of violence. Because of this, it's important to carefully test and consider the appropriate level of blocking required to support your key use cases and also minimize harm to end users.

Try a prompt that might trigger one of these categories:

In [12]:
impolite_prompt = "Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark:"

impolite_responses = model.generate_content(
    impolite_prompt,
    generation_config=generation_config,
    stream=True,
)

for response in impolite_responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "Oh"
    }
  }
}
usage_metadata {
}

candidates {
  content {
    role: "model"
    parts {
      text: ", the universe is testing us with stubbed toes now, is it? Here"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.09521484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1142578125
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1904296875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.09130859375
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.302734375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.07177734375
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.337890625
    severity: HARM_SE

#### Blocked responses

If the response is blocked, you will see that the final candidate includes `blocked: true`, and also observe which of the safety ratings triggered the blocking of the response (e.g. `finish_reason: SAFETY`).

In [13]:
rude_prompt = "Write a list of 5 very rude things that I might say to the universe after stubbing my toe in the dark:"

rude_responses = model.generate_content(
    rude_prompt,
    generation_config=generation_config,
    stream=True,
)

for response in rude_responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "As"
    }
  }
}
usage_metadata {
}

candidates {
  content {
    role: "model"
    parts {
      text: " an AI assistant programmed to be helpful and harmless, I cannot provide you with a"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.059326171875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.049560546875
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.07568359375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.02294921875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.1298828125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.040283203125
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.14257

### Defining thresholds for safety ratings

You may want to adjust the default safety filter thresholds depending on your business policies or use case. The Vertex AI Gemini API provides you a way to pass in a threshold for each category.

The list below shows the possible threshold labels:

* `BLOCK_ONLY_HIGH` - block when high probability of unsafe content is detected
* `BLOCK_MEDIUM_AND_ABOVE` - block when medium or high probability of content is detected
* `BLOCK_LOW_AND_ABOVE` - block when low, medium, or high probability of unsafe content is detected
* `BLOCK_NONE` - always show, regardless of probability of unsafe content

#### Set safety thresholds
Below, the safety thresholds have been set to the most sensitive threshold: `BLOCK_LOW_AND_ABOVE`

In [ ]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}

#### Test thresholds

Here you will reuse the impolite prompt from earlier together with the most sensitive safety threshold. It should block the response even with the `LOW` probability label.

In [14]:
impolite_prompt = "Write a list of 5 disrespectful things that I might say to the universe after stubbing my toe in the dark:"

impolite_responses = model.generate_content(
    impolite_prompt,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=True,
)

for response in impolite_responses:
    print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "Oh"
    }
  }
}
usage_metadata {
}

candidates {
  content {
    role: "model"
    parts {
      text: ", the universe is testing us with stubbed toes now, is it? Here"
    }
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.09521484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1142578125
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1904296875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.09130859375
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.302734375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.07177734375
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.337890625
    severity: HARM_SE

Let's look at how we understand block responses in the next section.

## Understanding Blocked Responses

The documentation for [`FinishReason`](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/GenerateContentResponse#finishreason) contains some more detailed explanations.

For example, the previous response was blocked with the `finish_reason: SAFETY`, indicating that
> The token generation was stopped as the response was flagged for safety reasons. NOTE: When streaming the `Candidate.content` will be empty if content filters blocked the output.

As of this writing, the table from the `FinishReason` have been reproduced below, but please look at the docs for the definitive explanations


Finish Reason | Explanation
--- | ---
`FINISH_REASON_UNSPECIFIED`	| The finish reason is unspecified.
`STOP`	| Natural stop point of the model or provided stop sequence.
`MAX_TOKENS`	| The maximum number of tokens as specified in the request was reached.
`SAFETY` |	The token generation was stopped as the response was flagged for safety reasons. NOTE: When streaming the `Candidate.content` will be empty if content filters blocked the output.
`RECITATION`	| The token generation was stopped as the response was flagged for unauthorized citations.
`OTHER`	All | other reasons that stopped the token generation
`BLOCKLIST` |	The token generation was stopped as the response was flagged for the terms which are included from the terminology blocklist.
`PROHIBITED_CONTENT`	| The token generation was stopped as the response was flagged for the prohibited contents.
`SPII`	| The token generation was stopped as the response was flagged for Sensitive Personally Identifiable Information (SPII) contents.